In [1]:
import cv2
import numpy as np
import time
import math

In [2]:
from ipynb.fs.full.HandTrackingModule import HandDetector

In [3]:
class Button():
    def __init__(self,x,y,w,h,value):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.value = value
        
    def draw(self,frame):
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,0,0),2)
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(200,200,200),-1)
        cv2.putText(frame,str(self.value),(self.x+10,self.y+50),0,1,(0,0,0),4)
        
    def isClicked(self,frame,x,y):
        if x >= self.x and x <= self.x + self.w and y >= self.y and y <= self.y + self.h :
                return True
        return False
    
    def click(self,frame):
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(0,0,0),2)
        cv2.rectangle(frame,(self.x,self.y),(self.x+self.w,self.y+self.h),(255,255,255),-1)
        cv2.putText(frame,str(self.value),(self.x+10,self.y+50),0,2,(0,0,0),4)
    
    def getValue(self,frame):
        return self.value
    
    

In [9]:
cap = cv2.VideoCapture(0)
cap.set(3,1920)
cap.set(4,720)
detector = HandDetector(detectionCon=0.8, maxHands=2)
button = Button(600,200,50,50,8)
buttonValues = [['1/x',"x2","del","AC"],
                ["7","8","9","+"],
                ["4","5","6","-"],
                ["1","2","3","*"],
                ["**","0","/","="]]
xOffset = 800
yOffset = 100
buttonList =  []
for i in range(len(buttonValues)):
    for j in range(len(buttonValues[i])):
        x = xOffset + j*75
        y = yOffset + i*75
        buttonList.append(Button(x,y,75,75,buttonValues[i][j]))

#print(buttonList[0])  
myEquation = ""
delay =0
error = False
while True:
    success,frame = cap.read()
    if not success:
        break
     
    frame = cv2.flip(frame,1)
    hands,frame = detector.findHands(frame)
    if not error:
        cv2.rectangle(frame,(800,40),(1100,100),(255,255,255),-1)
    else:
        cv2.rectangle(frame,(800,40),(1100,100),(200,200,255),-1)
        cv2.rectangle(frame,(800,40),(1100,100),(0,0,255),2)
    for button in buttonList:
        button.draw(frame)

    if hands:
        hand1 = hands[0]
        lmList = hand1["lmList"]
        x1,y1 = lmList[4][0],lmList[4][1]
        x2,y2 = lmList[8][0],lmList[8][1]
        cv2.circle(frame,(x1,y1),7,(255,0,0),-1)
        cv2.circle(frame,(x2,y2),7,(255,0,0),-1)
        cv2.line(frame,(x1,y1),(x2,y2),(255,0,0),3)
        length = math.hypot((x1-x2),(y1-y2))
        if length <= 40 and delay == 0 :
            for button in buttonList:
                if button.isClicked(frame,x1,y1) :
                    delay = 1
                    button.click(frame)
                    value = button.getValue(frame)
                    if value == "=":
                        try:
                            myEquation = str(round(eval(str(myEquation))))
                            error = False
                        except (NameError, SyntaxError,ZeroDivisionError):
                            error = True   
                    elif value == "del":
                        myEquation = str(myEquation[:-1])
                    elif value == "AC":
                        myEquation = ""
                    elif value == "1/x":
                        try:
                            myEquation = str(round(1/eval(str(myEquation)),4))
                            error = False
                        except (NameError, SyntaxError,ZeroDivisionError):
                            error = True      
                    elif value == "x2":
                        try:
                            myEquation = str(round(eval(str(myEquation)) * eval(str(myEquation))))
                            error = False
                        except (NameError, SyntaxError):
                            error = True       
                    else:
                        myEquation = myEquation + str(value)
                        
                        
    if delay !=0:
        delay += 1
        if delay > 15:
            delay = 0
        
        #thershold = 70
       
    #cv2.putText(frame,str(delay),(50,75),0,2,(0,0,0),2)
    cv2.putText(frame,str(myEquation),(800,75),0,1,(0,0,0),2)
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
  
cap.release()
cv2.destroyAllWindows()
